In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Carbon Emission Prediction Analysis - June 2025")
print("=" * 50)

# Generate synthetic carbon emission data
np.random.seed(42)
n_samples = 1000

# Create synthetic dataset
data = {
    'Year': np.random.randint(2000, 2025, n_samples),
    'GDP_per_capita': np.random.normal(35000, 15000, n_samples),
    'Population': np.random.normal(50000000, 30000000, n_samples),
    'Energy_consumption': np.random.normal(500, 200, n_samples),
    'Industrial_output': np.random.normal(1000, 400, n_samples),
    'Renewable_energy_percent': np.random.uniform(5, 80, n_samples),
    'Forest_coverage': np.random.uniform(10, 70, n_samples),
    'Urban_population_percent': np.random.uniform(20, 95, n_samples),
    'Country_type': np.random.choice(['Developed', 'Developing', 'Least_Developed'], n_samples),
    'Transportation_index': np.random.normal(100, 30, n_samples)
}

# Create target variable (carbon emissions) with realistic relationships
carbon_emissions = (
    data['GDP_per_capita'] * 0.0002 +
    data['Population'] * 0.00000001 +
    data['Energy_consumption'] * 2.5 +
    data['Industrial_output'] * 1.8 +
    data['Renewable_energy_percent'] * (-15) +
    data['Forest_coverage'] * (-8) +
    data['Urban_population_percent'] * 5 +
    data['Transportation_index'] * 3 +
    np.random.normal(0, 100, n_samples)
)

data['Carbon_emissions'] = np.maximum(carbon_emissions, 0)  # Ensure non-negative

# Create DataFrame
df = pd.DataFrame(data)

print(f"Dataset created with {len(df)} samples")
print("\nDataset Info:")
print(df.info())
print("\nFirst 5 rows:")
print(df.head())

# Statistical summary
print("\nStatistical Summary:")
print(df.describe())

# Check for missing values
print(f"\nMissing values: {df.isnull().sum().sum()}")

# Data Visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Carbon Emission Data Analysis', fontsize=16, fontweight='bold')

# Distribution of carbon emissions
axes[0, 0].hist(df['Carbon_emissions'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Distribution of Carbon Emissions')
axes[0, 0].set_xlabel('Carbon Emissions (Mt CO2)')
axes[0, 0].set_ylabel('Frequency')

# Carbon emissions by country type
df.boxplot(column='Carbon_emissions', by='Country_type', ax=axes[0, 1])
axes[0, 1].set_title('Carbon Emissions by Country Type')
axes[0, 1].set_xlabel('Country Type')
axes[0, 1].set_ylabel('Carbon Emissions (Mt CO2)')

# Correlation with GDP per capita
axes[0, 2].scatter(df['GDP_per_capita'], df['Carbon_emissions'], alpha=0.6, color='green')
axes[0, 2].set_title('Carbon Emissions vs GDP per Capita')
axes[0, 2].set_xlabel('GDP per Capita ($)')
axes[0, 2].set_ylabel('Carbon Emissions (Mt CO2)')

# Renewable energy impact
axes[1, 0].scatter(df['Renewable_energy_percent'], df['Carbon_emissions'], alpha=0.6, color='orange')
axes[1, 0].set_title('Carbon Emissions vs Renewable Energy %')
axes[1, 0].set_xlabel('Renewable Energy (%)')
axes[1, 0].set_ylabel('Carbon Emissions (Mt CO2)')

# Time series trend
yearly_emissions = df.groupby('Year')['Carbon_emissions'].mean()
axes[1, 1].plot(yearly_emissions.index, yearly_emissions.values, marker='o', linewidth=2, color='red')
axes[1, 1].set_title('Average Carbon Emissions by Year')
axes[1, 1].set_xlabel('Year')
axes[1, 1].set_ylabel('Average Carbon Emissions (Mt CO2)')

# Forest coverage impact
axes[1, 2].scatter(df['Forest_coverage'], df['Carbon_emissions'], alpha=0.6, color='purple')
axes[1, 2].set_title('Carbon Emissions vs Forest Coverage')
axes[1, 2].set_xlabel('Forest Coverage (%)')
axes[1, 2].set_ylabel('Carbon Emissions (Mt CO2)')

plt.tight_layout()
plt.show()

# Correlation matrix
plt.figure(figsize=(12, 8))
correlation_matrix = df.select_dtypes(include=[np.number]).corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={'label': 'Correlation'})
plt.title('Correlation Matrix of Variables', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Feature Engineering
# Encode categorical variables
le = LabelEncoder()
df['Country_type_encoded'] = le.fit_transform(df['Country_type'])

# Create additional features
df['Energy_per_capita'] = df['Energy_consumption'] / (df['Population'] / 1000000)
df['GDP_growth_proxy'] = df['GDP_per_capita'] * (df['Year'] - 2000) / 25
df['Environmental_score'] = (df['Renewable_energy_percent'] + df['Forest_coverage']) / 2

# Prepare features for modeling
feature_columns = ['Year', 'GDP_per_capita', 'Population', 'Energy_consumption', 
                  'Industrial_output', 'Renewable_energy_percent', 'Forest_coverage',
                  'Urban_population_percent', 'Transportation_index', 'Country_type_encoded',
                  'Energy_per_capita', 'GDP_growth_proxy', 'Environmental_score']

X = df[feature_columns]
y = df['Carbon_emissions']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

# Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0),
    'Lasso Regression': Lasso(alpha=1.0),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'Support Vector Regression': SVR(kernel='rbf', C=1.0)
}

# Train and evaluate models
results = {}
model_predictions = {}

print("\nModel Performance Comparison:")
print("=" * 60)

for name, model in models.items():
    if name == 'Support Vector Regression':
        # Use scaled features for SVR
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2,
        'CV_R2_mean': cv_scores.mean(),
        'CV_R2_std': cv_scores.std()
    }
    
    model_predictions[name] = y_pred
    
    print(f"{name}:")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  R²: {r2:.4f}")
    print(f"  CV R² (mean ± std): {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
    print()

# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(results).T
results_df = results_df.round(4)

print("Complete Results Summary:")
print(results_df)

# Visualize model performance
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Model Performance Analysis', fontsize=16, fontweight='bold')

# R² comparison
axes[0, 0].bar(results_df.index, results_df['R2'], color='skyblue', alpha=0.7)
axes[0, 0].set_title('R² Score Comparison')
axes[0, 0].set_ylabel('R² Score')
axes[0, 0].tick_params(axis='x', rotation=45)

# RMSE comparison
axes[0, 1].bar(results_df.index, results_df['RMSE'], color='lightcoral', alpha=0.7)
axes[0, 1].set_title('RMSE Comparison')
axes[0, 1].set_ylabel('RMSE')
axes[0, 1].tick_params(axis='x', rotation=45)

# Cross-validation scores
axes[1, 0].bar(results_df.index, results_df['CV_R2_mean'], 
               yerr=results_df['CV_R2_std'], color='lightgreen', alpha=0.7, capsize=5)
axes[1, 0].set_title('Cross-Validation R² Score')
axes[1, 0].set_ylabel('CV R² Score')
axes[1, 0].tick_params(axis='x', rotation=45)

# Prediction vs Actual for best model
best_model = results_df['R2'].idxmax()
best_predictions = model_predictions[best_model]
axes[1, 1].scatter(y_test, best_predictions, alpha=0.6, color='purple')
axes[1, 1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1, 1].set_title(f'Actual vs Predicted - {best_model}')
axes[1, 1].set_xlabel('Actual Carbon Emissions')
axes[1, 1].set_ylabel('Predicted Carbon Emissions')

plt.tight_layout()
plt.show()

print(f"\nBest performing model: {best_model}")
print(f"Best model R² score: {results_df.loc[best_model, 'R2']:.4f}")

# Feature importance for Random Forest
rf_model = models['Random Forest']
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['feature'], feature_importance['importance'], color='steelblue', alpha=0.7)
plt.title('Feature Importance - Random Forest Model', fontsize=14, fontweight='bold')
plt.xlabel('Importance')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print("\nFeature Importance (Random Forest):")
print(feature_importance)

# Hyperparameter tuning for the best model
if best_model == 'Random Forest':
    print("\nPerforming hyperparameter tuning for Random Forest...")
    
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    grid_search = GridSearchCV(
        RandomForestRegressor(random_state=42),
        param_grid,
        cv=5,
        scoring='r2',
        n_jobs=-1,
        verbose=1
    )
    
    grid_search.fit(X_train, y_train)
    
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best cross-validation score: {grid_search.best_score_:.4f}")
    
    # Train final model with best parameters
    final_model = grid_search.best_estimator_
    final_predictions = final_model.predict(X_test)
    final_r2 = r2_score(y_test, final_predictions)
    final_rmse = np.sqrt(mean_squared_error(y_test, final_predictions))
    
    print(f"Final model R² score: {final_r2:.4f}")
    print(f"Final model RMSE: {final_rmse:.2f}")

# Future predictions
print("\nGenerating future predictions...")

# Create future data (2025-2030)
future_years = np.arange(2025, 2031)
future_data = []

for year in future_years:
    # Assume some trends for future predictions
    sample_row = {
        'Year': year,
        'GDP_per_capita': 40000 + (year - 2025) * 1000,  # Growing GDP
        'Population': 60000000 + (year - 2025) * 1000000,  # Growing population
        'Energy_consumption': 500 + (year - 2025) * 10,  # Increasing energy use
        'Industrial_output': 1000 + (year - 2025) * 20,  # Growing industry
        'Renewable_energy_percent': 30 + (year - 2025) * 5,  # Increasing renewables
        'Forest_coverage': 40 - (year - 2025) * 0.5,  # Slight deforestation
        'Urban_population_percent': 65 + (year - 2025) * 1,  # Urbanization
        'Transportation_index': 100 + (year - 2025) * 2,  # Growing transportation
        'Country_type_encoded': 0  # Developed country
    }
    
    # Calculate engineered features
    sample_row['Energy_per_capita'] = sample_row['Energy_consumption'] / (sample_row['Population'] / 1000000)
    sample_row['GDP_growth_proxy'] = sample_row['GDP_per_capita'] * (sample_row['Year'] - 2000) / 25
    sample_row['Environmental_score'] = (sample_row['Renewable_energy_percent'] + sample_row['Forest_coverage']) / 2
    
    future_data.append(sample_row)

future_df = pd.DataFrame(future_data)
future_X = future_df[feature_columns]

# Make predictions
future_predictions = final_model.predict(future_X)

# Visualize future predictions
plt.figure(figsize=(12, 6))
plt.plot(future_years, future_predictions, marker='o', linewidth=2, markersize=8, color='red')
plt.title('Predicted Carbon Emissions (2025-2030)', fontsize=14, fontweight='bold')
plt.xlabel('Year')
plt.ylabel('Predicted Carbon Emissions (Mt CO2)')
plt.grid(True, alpha=0.3)
plt.xticks(future_years)
plt.tight_layout()
plt.show()

print("\nFuture Predictions:")
for year, pred in zip(future_years, future_predictions):
    print(f"Year {year}: {pred:.2f} Mt CO2")

# Model insights and recommendations
print("\nModel Insights and Recommendations:")
print("=" * 50)
print("1. Key factors affecting carbon emissions:")
for i, row in feature_importance.head(5).iterrows():
    print(f"   - {row['feature']}: {row['importance']:.3f}")

print("\n2. Policy recommendations:")
print("   - Increase renewable energy adoption")
print("   - Improve energy efficiency in industry")
print("   - Implement carbon pricing mechanisms")
print("   - Promote forest conservation")
print("   - Develop cleaner transportation systems")

print("\n3. Model limitations:")
print("   - Based on synthetic data for demonstration")
print("   - Real-world factors may be more complex")
print("   - External events (policy changes, crises) not captured")
print("   - Temporal dependencies could be better modeled")

print("\nAnalysis completed successfully!")